In [3]:
from helper import *

In [4]:
df_graph = pickle.load(open('../data/random_graph/ucla_20walk_graph.pkl', 'rb'))
df_dynamics = pd.read_csv("../data/dynamics_pairs/fca_k8_ucla_20walk_dynamics.csv")
y = df_dynamics.y
base = df_dynamics.baseline_width
df_dynamics = df_dynamics.loc[:, 's1_1':'s50_20']
df = pd.read_csv("../data/dynamics_pairs/fca_k8_ucla_20walk_colored_adj.csv")
df_feature = pd.read_csv("../data/random_graph/ucla_20walk_graph_features.csv")

In [6]:
scale = 4 # scale down colored adj
X = pd.concat([pd.DataFrame(df_graph.T), df/scale], axis=1)

In [17]:
X.shape

(10000, 20400)

In [9]:
W = dict_handcraft(df_feature.diameter, base, X, 4)

In [10]:
from sklearn.linear_model import LogisticRegression   
Y_data = y
under_sampler = RandomUnderSampler()
X_res, y_res = under_sampler.fit_resample(X.values, Y_data)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, 
                                                    test_size = 0.2, 
                                                    random_state = 4, 
                                                    stratify = y_res)
logreg = LogisticRegression()  
logreg.fit(np.matmul(W, X_train.T).T, y_train)  
#Y_pred = logreg.predict(X[:2, :])  
#Y_predict = logreg.predict_proba(X[:2, :])   
score = logreg.score(np.matmul(W, X_test.T).T, y_test)  
print(score)  

0.7517875383043923


In [20]:
print(np.exp(logreg.coef_))#/(1+np.exp(logreg.coef_)))
#add p-value
#random sample
#print(logreg.intercept_)

[[1.53018233 1.77966321 1.23387545 1.56172237 0.26692042 0.43189418
  0.16594548 0.39250786 3.38329152 1.78204565 3.00108536 1.41181359]]
